Notes and Examples from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


# Exploratory Data Analysis

## A Statistical Approach

To address the limitations of anecdotes, we will use the tools of statistics, which include:

* Data collection

We will use data from a large national survey that was designed explicitly with the goal of generating statistically valid inferences about the U.S. population.

* Descriptive statistics

We will generate statistics that summarize the data concisely, and evaluate different ways to visualize data.

* Exploratory data analysis

We will look for patterns, differences, and other features that address the questions we are interested in. At the same time we will check for inconsistencies and identify limitations.

* Estimation

We will use data from a sample to estimate characteristics of the general population.

* Hypothesis testing

Where we see apparent effects, like a difference between two groups, we will evaluate whether the effect might have happened by chance.

## Data

**The National Survey of Family Growth**

* Check how the data is collected which can be;
    * a cross-sectional study, which means that it captures a snapshot of a group at a point in time
    * a longitudinal study, which observes a group repeatedly over a period of time.

* Define the targeted population of the study, which is NSFG people in US aged 15-44 in this analysis. 
* Make sure that the collected data consists of representative sample (means every respondent has equal chance to join to the survey).

*p.s: The NSFG is not representative; instead it is deliberately oversampled as explained in the book.*

In [42]:
from __future__ import print_function, division

import nsfg
import numpy as np
import pandas as pd

In [43]:
df = nsfg.ReadFemPreg()
df.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,9.1250
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,7.0000
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,6.1875


In [44]:
df.columns

Index(['caseid', 'pregordr', 'howpreg_n', 'howpreg_p', 'moscurrp', 'nowprgdk',
       'pregend1', 'pregend2', 'nbrnaliv', 'multbrth',
       ...
       'laborfor_i', 'religion_i', 'metro_i', 'basewgt', 'adj_mod_basewgt',
       'finalwgt', 'secu_p', 'sest', 'cmintvw', 'totalwgt_lb'],
      dtype='object', length=244)

### Variables
* caseid is the integer ID of the respondent.
* prglength is the integer duration of the pregnancy in weeks.
* outcome is an integer code for the outcome of the pregnancy. The code 1 indicates a live birth.
* pregordr is a pregnancy serial number; for example, the code for a respondent’s first pregnancy is 1, for the second pregnancy is 2, and so on.
* birthord is a serial number for live births; the code for a respondent’s first child is 1, and so on. For outcomes other than live births, this field is blank.
* birthwgt_lb and birthwgt_oz contain the pounds and ounces parts of the birth weight of the baby.
* agepreg is the mother’s age at the end of the pregnancy.
* finalwgt is the statistical weight associated with the respondent. It is a floating-point value that indicates the number of people in the U.S. population this respondent represents.

### Transformation

The file (module) nsfg.py includes CleanFemPreg() function to do the required cleaning.

In [45]:
def CleanFemPreg(df):
    
    #convert the mothers' age from centiyears to years
    df.agepreg /= 100.0
    
    # birthwgt_lb and birthwgt_oz includes codes as values like 97, 98,99 instead of measurements
    na_vals = [97,98,99]
    df.birthwgt_lb.replace(na_vals, np.nan, inplace=True)
    df.birthwgt_oz.replace(na_vals, np.nan, inplace=True)
    
    # crete a new column totalwgt_lb that combines pounds and ounces into a single quantity, in pounds
    df['totalwgt_lb'] = df.birthwgt_lb + df.birthwgt_oz / 16.0
    
    return df

### Data Validation

Data validation is useful to understand the data and save time from further data misunderstandings, corrections.
One basic way is to get familiar with the data is looking to the basic statistics.

In [46]:
df.outcome.value_counts().sort_index()

1    9148
2    1862
3     120
4    1921
5     190
6     352
Name: outcome, dtype: int64

In [47]:
df.birthwgt_lb.value_counts(sort=False)

8.0     1889
7.0     3049
6.0     2223
4.0      229
5.0      697
10.0     132
12.0      10
14.0       3
3.0       98
1.0       40
2.0       53
0.0        8
9.0      623
11.0      26
13.0       3
15.0       1
Name: birthwgt_lb, dtype: int64

#### Interpretation

Working with data requires two different aspects to consider; 
* the level of statistics
* the level of context

As an example, if we want to look at the sequence of outcomes for a few respondents, we need to do some data processing because of the nature of this data set. 

In [48]:
def MakePregMap(df):
    d = defaultdict(list)
    for index, caseid in df.caseid.iteritems():
        d[caseid].append(index)
    return d
    

In [49]:
from collections import defaultdict
preg_map = MakePregMap(df)

In [50]:
caseid=10229
indices = preg_map[caseid]

In [51]:
df.outcome[indices].values

array([4, 4, 4, 4, 4, 4, 1])

The outcome code 1 indicates a live birth. Code 4 indicates a miscarriage; that is, a pregnancy that ended spontaneously, usually with no known medical cause.

But remembering the context, this data tells the story of a woman who was pregnant six times, each time ending in miscarriage. Her seventh and most recent pregnancy ended in a live birth. If we consider this data with empathy, it is natural to be moved by the story it tells.

# Exercises

Select the `birthord` column, print the value counts, and compare to results published in the [codebook](http://www.icpsr.umich.edu/nsfg6/Controller?displayPage=labelDetails&fileCode=PREG&section=A&subSec=8016&srtLabel=611933)

In [52]:
df.birthord.value_counts()

1.0     4413
2.0     2874
3.0     1234
4.0      421
5.0      126
6.0       50
7.0       20
8.0        7
9.0        2
10.0       1
Name: birthord, dtype: int64

In [53]:
df.birthord.isnull().sum()

4445

Select the `prglngth` column, print the value counts, and compare to results published in the [codebook](http://www.icpsr.umich.edu/nsfg6/Controller?displayPage=labelDetails&fileCode=PREG&section=A&subSec=8016&srtLabel=611931)

In [54]:
df.prglngth.value_counts()

39    4744
40    1120
38     609
9      594
41     591
6      543
37     457
13     446
4      412
8      409
35     357
36     329
42     328
17     253
11     202
30     198
5      181
7      175
12     170
3      151
43     148
22     147
10     137
32     122
26     117
2       78
34      60
33      50
44      46
16      44
15      39
28      38
21      37
19      34
24      31
31      29
14      29
29      23
20      18
18      17
0       15
25      15
23      12
45      10
1        9
27       8
48       7
50       2
46       1
47       1
Name: prglngth, dtype: int64

Create a new column named <tt>totalwgt_kg</tt> that contains birth weight in kilograms.  Compute its mean.  Remember that when you create a new column, you have to use dictionary syntax, not dot notation.

In [56]:
df = CleanFemPreg(df)
df.columns

Index(['caseid', 'pregordr', 'howpreg_n', 'howpreg_p', 'moscurrp', 'nowprgdk',
       'pregend1', 'pregend2', 'nbrnaliv', 'multbrth',
       ...
       'laborfor_i', 'religion_i', 'metro_i', 'basewgt', 'adj_mod_basewgt',
       'finalwgt', 'secu_p', 'sest', 'cmintvw', 'totalwgt_lb'],
      dtype='object', length=244)

In [57]:
df['totalwgt_kg'] = df.totalwgt_lb * 0.028349523125

In [58]:
df.columns

Index(['caseid', 'pregordr', 'howpreg_n', 'howpreg_p', 'moscurrp', 'nowprgdk',
       'pregend1', 'pregend2', 'nbrnaliv', 'multbrth',
       ...
       'religion_i', 'metro_i', 'basewgt', 'adj_mod_basewgt', 'finalwgt',
       'secu_p', 'sest', 'cmintvw', 'totalwgt_lb', 'totalwgt_kg'],
      dtype='object', length=245)

In [59]:
resp = nsfg.ReadFemResp()

In [60]:
resp.head()

,caseid,rscrinf,rdormres,rostscrn,rscreenhisp,rscreenrace,age_a,age_r,cmbirth,agescrn,...,pubassis_i,basewgt,adj_mod_basewgt,finalwgt,secu_r,sest,cmintvw,cmlstyr,screentime,intvlngth
0,2298,1,5,5,1,5.0,27,27,902,27,...,0,3247.916977,5123.759559,5556.717241,2,18,1234,1222,18:26:36,110.492667
1,5012,1,5,1,5,5.0,42,42,718,42,...,0,2335.279149,2846.799490,4744.191350,2,18,1233,1221,16:30:59,64.294000
2,11586,1,5,1,5,5.0,43,43,708,43,...,0,2335.279149,2846.799490,4744.191350,2,18,1234,1222,18:19:09,75.149167
3,6794,5,5,4,1,5.0,15,15,1042,15,...,0,3783.152221,5071.464231,5923.977368,2,18,1234,1222,15:54:43,28.642833
4,616,1,5,4,1,5.0,20,20,991,20,...,0,5341.329968,6437.335772,7229.128072,2,18,1233,1221,14:19:44,69.502667


Select the `age_r` column from `resp` and print the value counts.  How old are the youngest and oldest respondents?

In [61]:
resp.age_r.value_counts().sort_index()

15    217
16    223
17    234
18    235
19    241
20    258
21    267
22    287
23    282
24    269
25    267
26    260
27    255
28    252
29    262
30    292
31    278
32    273
33    257
34    255
35    262
36    266
37    271
38    256
39    215
40    256
41    250
42    215
43    253
44    235
Name: age_r, dtype: int64

We can use the `caseid` to match up rows from `resp` and `preg`.  For example, we can select the row from `resp` for `caseid` 2298 like this:

In [64]:
resp[df.caseid==2298]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,caseid,rscrinf,rdormres,rostscrn,rscreenhisp,rscreenrace,age_a,age_r,cmbirth,agescrn,...,pubassis_i,basewgt,adj_mod_basewgt,finalwgt,secu_r,sest,cmintvw,cmlstyr,screentime,intvlngth
2610,8475,1,5,3,5,5.0,37,37,785,37,...,0,3453.674329,4240.287846,7738.101291,1,25,1235,1223,15:17:16,57.485833
2611,3466,1,5,4,1,5.0,32,32,847,32,...,0,4307.168559,5168.443791,6559.272015,1,25,1231,1219,18:32:56,96.039167
2612,10250,5,5,4,5,2.0,31,31,859,31,...,0,3453.674329,4176.212214,7245.728191,1,25,1234,1222,19:16:29,91.277000
2613,8030,1,5,2,1,5.0,35,35,817,35,...,0,4600.820609,8736.684047,9450.471134,2,25,1238,1226,18:55:58,92.141167


How old is the respondent with `caseid` 1?

In [65]:
resp[df.caseid==1].age_r

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


0    27
1    42
Name: age_r, dtype: int64

What are the pregnancy lengths for the respondent with `caseid` 2298?

In [69]:
df[df.caseid==2298].prglngth

2610    40
2611    36
2612    30
2613    40
Name: prglngth, dtype: int64